<a href="https://colab.research.google.com/github/Florencemon/1erpagweb/blob/main/KNN_K_VECINOS_M%C3%81S_CERCANOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Librerias para el manejo de datos:
import pandas as pd
import numpy as np
# Librerias de sklearn:
from sklearn.model_selection import train_test_split # Libreria para partir en entrenamiento y prueba:
from sklearn.neighbors import KNeighborsClassifier # Libreria para ajustar el modelo: Clasificador basado en el algoritmo de k-vecinos más cercanos.
from sklearn.model_selection import GridSearchCV # Libreria para cross validation: Sirve para asegurarse de que el modelo no esté "sobreajustado" (overfitted) al conjunto de entrenamiento
from sklearn.impute import SimpleImputer # Libreria para importar valores faltantes
from sklearn.preprocessing import StandardScaler # Libreria para estandarizar datos. Estandariza las variables: les da media 0 y desviación estándar 1. Muy útil para modelos sensibles a escala (como KNN o regresión logística)
from sklearn.pipeline import Pipeline # organizar pasos de preprocesamiento y modelado de forma estructurada y segura



In [2]:
# Datos
url = "https://raw.githubusercontent.com/TruePos/videos/refs/heads/master/KNN/train.csv"
baseball = pd.read_csv(url)
baseball = baseball[["plate_x",
                     "plate_z",
                     "launch_angle",
                     "launch_speed",
                     "is_home_run"]]
baseball.head()



,plate_x,plate_z,launch_angle,launch_speed,is_home_run
0,-0.15,1.97,9.0,108.9,0
1,-0.71,2.88,18.0,NaN,0
2,-0.54,1.75,49.0,75.0,0
3,-0.42,3.15,54.0,90.6,0
4,0.18,3.27,6.0,NaN,0


In [3]:
#---------------------- Separacion inicial ----------------------#

# Separar variables predictoras y objetivo

# Variable objetivo = y
# Variable predictora = x

X = baseball.drop(columns='is_home_run')
y = baseball['is_home_run'].values

# Separar en entrenamiento y prueba las variables predictoras y objetivos.
# 80% para entrenar y 20% pruebas
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=2352,
    stratify=y
)


In [4]:
#---------------------- Modelacion base KNN----------------------#

# Definicion del modelo de KNN contemplando a los 10 'K' vecinos mas cercanos
knn_model = KNeighborsClassifier(n_neighbors = 10,
                                 weights = 'uniform', # weights = 'uniform' que es el parametro normal
                                 p = 1) #1 la distancia de Manhattan

# definimos los metodos para realizar el Preprocesamiento
imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()

# juntamos todo en un Pipeline.
pipe = Pipeline(steps=[
    ("Imputacion", imputer),
    ("Scaler", scaler),
    ("knn", knn_model)
])

# datos de Entrenamiento del pipeline
pipe.fit(X_train, y_train)


Pipeline(steps=[('Imputacion', SimpleImputer()), ('Scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=10, p=1))])

In [5]:
# Prediccion en test: obtenemos las predicciones en el conjunto de prueba (test)
y_pred = pipe.predict(X_test)

# calculamos el Accuracy : que es de 0.9563
accuracy = pipe.score(X_test, y_test)
print(f"Accuracy sin optimización: {accuracy:.4f}")


Accuracy sin optimización: 0.9563


In [6]:
#----------- Optimizacion de hiperparametros -----------#

# Modelo definimos el modelo sin especificar sus parametros (Modelo sin parámetros definidos)
knn = KNeighborsClassifier()

# Nuevo pipeline: lo añadimos al Pipeline

pipe = Pipeline(steps=[
    ("Imputacion", imputer),
    ("Scaler", scaler),
    ("knn", knn)
])

# Diccionario de hiperparámetros: especificamos los valores a evaluar de los hiperparámetros en un diccionario

param_grid = {
    "knn__n_neighbors": np.arange(1, 15), #es importante poner antes del nombre del parametro'knn' seguido de dos guiones bajos
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2]
}

# GridSearchCV
knn_search = GridSearchCV(pipe, param_grid, cv=5) # usamos la funcion 'GridSearchCV' para Optimizar el pipeline
knn_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Imputacion', SimpleImputer()),
                                       ('Scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']})

In [13]:
mejores_params = knn_search.best_params_
# Convertimos el tipo si es necesario
mejores_params['knn__n_neighbors'] = int(mejores_params['knn__n_neighbors'])

print(f"""
✅ Mejor modelo encontrado:
- n_neighbors: {int(mejores_params['knn__n_neighbors'])}
- weights: {mejores_params['knn__weights']}
- p (distancia): {mejores_params['knn__p']}
🏆 Accuracy (CV): {knn_search.best_score_:.4f}
""")



✅ Mejor modelo encontrado:
- n_neighbors: 13
- weights: uniform
- p (distancia): 1
🏆 Accuracy (CV): 0.9588

